In [ ]:
!pip install -U gast==0.2.2

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Load data

train = pd.read_csv('./data/train_wj.csv') # load train data
test = pd.read_csv('./data/test_wj.csv') # load test data

In [ ]:
train = pd.get_dummies(train, columns=["dow"], prefix="dow", drop_first=True)
test = pd.get_dummies(test, columns=["dow"], prefix="dow", drop_first=True)

In [ ]:
# 모델에 넣을 데이터 만들기

test_cols = test.columns.values
X_cols_rm = ["n_id", "id", "date", "title", "content", "info", "content_tokens", "title_tokens"]
X_cols = test_cols[np.isin(test_cols, X_cols_rm, invert = True)]
X_test = test.loc[:, X_cols]

In [ ]:
# Split data into x and y

train_cols = train.columns.values
X_cols_rm = ["date", "breakfast", "lunch", "dinner", "lunch_n", "dinner_n", "lunch_rate", "dinner_rate"]
X_cols = train_cols[np.isin(train_cols, X_cols_rm, invert = True)]
y_cols = ["lunch_n", "dinner_n"]

X_train = train.loc[:, X_cols] # input of train data
y_train = train.loc[:, y_cols] # output of train data
X_test = test.loc[:, X_cols]   # input of test data

In [ ]:
# Split train data into train data and valid data

train_sample, valid = train_test_split(train, test_size=0.2) # randomly sample 80% of train data

In [ ]:
# Split data into x and y

X_train_sample = train_sample.loc[:, X_cols]  # input of train_sample data
y_train_sample = train_sample.loc[:, y_cols]  # output of train_sample data
X_valid = valid.loc[:, X_cols]                # input of valid data
y_valid = valid.loc[:, y_cols]                # output of valid data

In [ ]:
# Define Keras model

model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=len(y_cols), activation='linear'))

In [ ]:
# Compile Keras model

model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [ ]:
# Fit Keras model

epochs = 100
batch = 50
valid_split = 0.1

model.fit(X_train_sample,
          y_train_sample, 
          epochs=epochs, 
          batch_size=batch, 
          validation_split=valid_split)

In [ ]:
# Evaluate Keras model

model.evaluate(X_valid, y_valid)

In [ ]:
# Predict Keras model

y_test = model.predict(X_test)

In [ ]:
y_test

In [ ]:
# Make submission file 

sub_index = pd.read_csv('sample_submission.csv', index_col=0)
submission = sub_index + test_Y
submission.to_csv('submission1_0129.csv')